# Day11_2 별첨: 고급 감성 분석 (LLM & Embedding)

## 📚 학습 목표

**Part 1: LLM 기반 감성 분석**
1. Google Gemini API 설정 및 사용법 익히기
2. Zero-shot 감성 분류 구현하기
3. Few-shot 프롬프트 엔지니어링 이해하기
4. 배치 처리로 여러 텍스트 분석하기
5. LLM 감성 분석 결과 평가하기

**Part 2: 임베딩 기반 감성 분석**
1. 텍스트 임베딩 개념 이해하기
2. Google Gemini Embedding API 활용하기
3. sentence-transformers로 로컬 임베딩 생성하기
4. 코사인 유사도 기반 감성 분류 구현하기

---

## 🎯 왜 이것을 배우나요?

| 방법 | 장점 | 단점 | 적합한 상황 |
|------|------|------|------------|
| 사전 기반 | 빠름, 무료, 해석 쉬움 | 도메인 한정적 | 초기 분석, 알려진 도메인 |
| LLM Zero-shot | 라벨 불필요, 유연함 | API 비용, 속도 | 다양한 도메인, 새로운 표현 |
| LLM Few-shot | 높은 정확도, 커스텀 | 예제 준비 필요 | 특정 도메인, 세밀한 분류 |
| 임베딩 기반 | 의미 기반 분류, 대량 처리 | 기준 임베딩 필요 | 대량 분류, 유사 문서 검색 |

**분석가 관점**: LLM과 임베딩은 사전에 없는 새로운 표현도 이해할 수 있어 실무에서 점점 더 중요해지고 있습니다!

---

# Part 1: LLM 기반 감성 분석

---

## 1.1 Google Gemini API 설정

### API 키 발급 방법

1. [Google AI Studio](https://aistudio.google.com/) 접속
2. Google 계정으로 로그인
3. "Get API key" 클릭
4. "Create API key" 선택
5. 생성된 키를 복사하여 환경변수로 설정

### 무료 한도

- **gemini-2.0-flash**: 분당 15 요청, 일일 1,500 요청
- **gemini-embedding-001**: 월 100만 토큰 무료

> ⚠️ **주의**: API 키는 절대 코드에 직접 넣지 마세요! 환경변수로 관리하세요.

In [ ]:
# 필요한 라이브러리 설치 (최초 1회)
# !pip install google-genai pandas plotly

In [ ]:
# 라이브러리 임포트
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

# Google Gemini API
try:
    from google import genai
    from google.genai import types
    gemini_available = True
    print("Google Gemini API 로드 완료")
except ImportError:
    gemini_available = False
    print("google-genai 미설치. pip install google-genai 로 설치하세요.")

print("라이브러리 임포트 완료")

In [ ]:
# API 클라이언트 설정
# 환경변수에서 API 키 가져오기
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

if GOOGLE_API_KEY and gemini_available:
    client = genai.Client(api_key=GOOGLE_API_KEY)
    print("Gemini API 클라이언트 설정 완료")
else:
    client = None
    print("API 키가 설정되지 않았거나 라이브러리가 없습니다.")
    print("환경변수 설정: export GOOGLE_API_KEY='your-api-key'")

---

## 1.2 Zero-shot 감성 분류

### Zero-shot이란?

**Zero-shot Learning**은 사전 예제 없이 모델이 새로운 작업을 수행하는 것입니다.

LLM에게 "이 텍스트의 감성을 분석해줘"라고 요청하면, 모델이 학습된 일반 지식으로 분류합니다.

### 장단점

| 장점 | 단점 |
|------|------|
| 예제 데이터 불필요 | 복잡한 분류에 부정확할 수 있음 |
| 빠른 프로토타이핑 | 도메인 특화 용어에 약함 |
| 새로운 도메인에 바로 적용 | 일관성이 떨어질 수 있음 |

In [ ]:
def analyze_sentiment_zero_shot(text):
    """
    Zero-shot 방식으로 감성 분석
    
    Parameters:
    - text: 분석할 텍스트
    
    Returns:
    - dict: 감성(긍정/부정/중립), 신뢰도, 설명
    """
    if not client:
        return {"sentiment": "unknown", "confidence": 0, "reason": "API 미설정"}
    
    prompt = f"""다음 텍스트의 감성을 분석해주세요.

텍스트: {text}

다음 형식으로 답변해주세요:
감성: [긍정/부정/중립]
신뢰도: [높음/중간/낮음]
이유: [한 문장으로 설명]"""
    
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash-exp",
            contents=prompt
        )
        result_text = response.text
        
        # 결과 파싱
        lines = result_text.strip().split('\n')
        result = {"raw": result_text}
        
        for line in lines:
            if '감성:' in line:
                result['sentiment'] = line.split(':')[1].strip()
            elif '신뢰도:' in line:
                result['confidence'] = line.split(':')[1].strip()
            elif '이유:' in line:
                result['reason'] = line.split(':')[1].strip()
        
        return result
    except Exception as e:
        return {"sentiment": "error", "error": str(e)}

# 테스트 (API 키가 있을 때만 실행)
if client:
    test_texts = [
        "이 제품 정말 좋아요! 배송도 빠르고 품질도 최고예요.",
        "돈 아까워요. 완전 실망입니다. 다시는 안 살 거예요.",
        "그냥 그래요. 기대했던 것과 비슷해요."
    ]
    
    print("Zero-shot 감성 분석 결과")
    print("=" * 60)
    
    for text in test_texts:
        result = analyze_sentiment_zero_shot(text)
        print(f"\n텍스트: {text}")
        print(f"  감성: {result.get('sentiment', 'N/A')}")
        print(f"  신뢰도: {result.get('confidence', 'N/A')}")
        print(f"  이유: {result.get('reason', 'N/A')}")
        time.sleep(1)  # API 속도 제한 대비
else:
    print("API가 설정되지 않아 테스트를 건너뜁니다.")

### 💡 실무 예시: 다양한 Zero-shot 프롬프트

In [ ]:
def analyze_sentiment_simple(text):
    """간단한 Zero-shot 프롬프트 (3분류)"""
    if not client:
        return "API 미설정"
    
    prompt = f"""다음 텍스트의 감성을 '긍정', '부정', '중립' 중 하나로만 답하세요.
텍스트: {text}
감성:"""
    
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt
    )
    return response.text.strip()


def analyze_sentiment_scale(text):
    """5점 척도 Zero-shot 프롬프트"""
    if not client:
        return "API 미설정"
    
    prompt = f"""다음 텍스트의 감성 점수를 1~5 사이의 숫자로만 답하세요.
1: 매우 부정, 2: 부정, 3: 중립, 4: 긍정, 5: 매우 긍정

텍스트: {text}
점수:"""
    
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt
    )
    return response.text.strip()


def analyze_sentiment_detailed(text):
    """상세 분석 Zero-shot 프롬프트"""
    if not client:
        return "API 미설정"
    
    prompt = f"""다음 고객 리뷰를 분석해주세요.

리뷰: {text}

JSON 형식으로 답변해주세요:
{{
  "overall_sentiment": "긍정/부정/중립",
  "aspects": [
    {{"aspect": "언급된 측면", "sentiment": "긍정/부정/중립"}}
  ],
  "keywords": ["핵심 키워드들"],
  "summary": "한 줄 요약"
}}"""
    
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt
    )
    return response.text.strip()


# 테스트
if client:
    test_review = "배송은 빨랐는데 제품 품질이 별로예요. 가격 대비 실망입니다."
    
    print("다양한 Zero-shot 프롬프트 비교")
    print("=" * 60)
    print(f"\n리뷰: {test_review}")
    print(f"\n[간단한 분류] {analyze_sentiment_simple(test_review)}")
    time.sleep(1)
    print(f"\n[5점 척도] {analyze_sentiment_scale(test_review)}")
    time.sleep(1)
    print(f"\n[상세 분석]\n{analyze_sentiment_detailed(test_review)}")
else:
    print("API가 설정되지 않아 테스트를 건너뜁니다.")

---

## 1.3 Few-shot 프롬프트 엔지니어링

### Few-shot이란?

**Few-shot Learning**은 몇 개의 예제를 제공하여 모델이 패턴을 학습하게 하는 방식입니다.

### Zero-shot vs Few-shot

| 구분 | Zero-shot | Few-shot |
|------|-----------|----------|
| 예제 | 없음 | 2~5개 |
| 정확도 | 보통 | 높음 |
| 일관성 | 낮음 | 높음 |
| 토큰 사용 | 적음 | 많음 |
| 커스터마이징 | 제한적 | 유연함 |

In [ ]:
def analyze_sentiment_few_shot(text, examples=None):
    """
    Few-shot 방식으로 감성 분석
    
    Parameters:
    - text: 분석할 텍스트
    - examples: 예제 리스트 [(텍스트, 감성), ...]
    
    Returns:
    - str: 감성 분류 결과
    """
    if not client:
        return "API 미설정"
    
    # 기본 예제
    if examples is None:
        examples = [
            ("정말 만족합니다! 최고의 제품이에요.", "긍정"),
            ("완전 실망이에요. 다시는 안 살 거예요.", "부정"),
            ("보통이에요. 특별히 좋지도 나쁘지도 않아요.", "중립"),
            ("가격이 좀 비싸긴 한데 품질은 좋아요.", "긍정"),
            ("배송은 빨랐는데 제품이 기대에 못 미쳐요.", "부정")
        ]
    
    # 프롬프트 구성
    prompt = """다음은 고객 리뷰의 감성 분류 예시입니다.

"""
    
    for ex_text, ex_sentiment in examples:
        prompt += f"리뷰: {ex_text}\n감성: {ex_sentiment}\n\n"
    
    prompt += f"""위의 예시를 참고하여 다음 리뷰의 감성을 분류해주세요.

리뷰: {text}
감성:"""
    
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt
    )
    return response.text.strip()


# 테스트
if client:
    test_texts = [
        "사이즈가 딱 맞아요! 색상도 예쁘고 만족합니다.",
        "한 달 만에 고장났어요. 환불 받고 싶어요.",
        "가성비는 괜찮은 것 같아요.",
        "배송이 너무 느려요. 2주나 걸렸어요."
    ]
    
    print("Few-shot 감성 분석 결과")
    print("=" * 60)
    
    for text in test_texts:
        result = analyze_sentiment_few_shot(text)
        print(f"\n리뷰: {text}")
        print(f"  감성: {result}")
        time.sleep(1)
else:
    print("API가 설정되지 않아 테스트를 건너뜁니다.")

### 💡 실무 예시: 도메인 특화 Few-shot

In [ ]:
# 음식점 리뷰 특화 예제
food_review_examples = [
    ("맛있어요! 양도 많고 가격도 적당해요.", "긍정"),
    ("너무 짜고 느끼해요. 위생도 별로인 것 같아요.", "부정"),
    ("평범해요. 특별히 맛있지도 맛없지도 않아요.", "중립"),
    ("음식은 맛있는데 서비스가 불친절해요.", "중립"),  # 복합 감성
    ("웨이팅이 길었지만 기다린 보람이 있어요!", "긍정")
]

# 전자제품 리뷰 특화 예제
electronics_examples = [
    ("성능 좋고 발열도 적어요. 가성비 최고!", "긍정"),
    ("한 달 만에 고장났어요. AS도 별로예요.", "부정"),
    ("평범한 스펙이에요. 가격 대비 무난합니다.", "중립"),
    ("디자인은 예쁜데 배터리가 너무 빨리 닳아요.", "부정"),
    ("처음엔 불안했는데 써보니 좋아요!", "긍정")
]

if client:
    # 음식점 리뷰 테스트
    food_review = "국물이 진하고 면발이 쫄깃해요. 재방문 의사 100%!"
    result = analyze_sentiment_few_shot(food_review, food_review_examples)
    print(f"[음식점 리뷰] {food_review}")
    print(f"  감성: {result}")
    
    time.sleep(1)
    
    # 전자제품 리뷰 테스트
    electronics_review = "화면은 선명한데 스피커 음질이 아쉬워요."
    result = analyze_sentiment_few_shot(electronics_review, electronics_examples)
    print(f"\n[전자제품 리뷰] {electronics_review}")
    print(f"  감성: {result}")
else:
    print("API가 설정되지 않아 테스트를 건너뜁니다.")

---

## 1.4 배치 처리 및 결과 분석

In [ ]:
def analyze_batch(texts, method="zero-shot", delay=1):
    """
    여러 텍스트를 배치로 감성 분석
    
    Parameters:
    - texts: 텍스트 리스트
    - method: "zero-shot" 또는 "few-shot"
    - delay: API 호출 간 대기 시간 (초)
    
    Returns:
    - DataFrame: 분석 결과
    """
    results = []
    
    for i, text in enumerate(texts):
        print(f"처리 중... {i+1}/{len(texts)}", end="\r")
        
        if method == "few-shot":
            sentiment = analyze_sentiment_few_shot(text)
        else:
            sentiment = analyze_sentiment_simple(text)
        
        results.append({
            "text": text,
            "sentiment": sentiment
        })
        
        if i < len(texts) - 1:
            time.sleep(delay)
    
    print("\n분석 완료!")
    return pd.DataFrame(results)


# 샘플 리뷰 데이터
sample_reviews = [
    "배송 빠르고 포장 꼼꼼해요. 만족합니다!",
    "사진과 너무 다르게 생겼어요. 실망입니다.",
    "가격 대비 무난해요.",
    "품질 좋아요! 재구매 의사 있어요.",
    "교환 요청했는데 응대가 불친절해요.",
    "색상이 생각보다 예뻐요.",
    "한 달 써봤는데 아직 잘 모르겠어요.",
    "최악이에요. 환불 받았어요."
]

if client:
    # 배치 분석 실행
    results_df = analyze_batch(sample_reviews, method="few-shot")
    print("\n분석 결과:")
    print(results_df)
else:
    print("API가 설정되지 않았습니다.")
    # 시뮬레이션 데이터
    results_df = pd.DataFrame({
        "text": sample_reviews,
        "sentiment": ["긍정", "부정", "중립", "긍정", "부정", "긍정", "중립", "부정"]
    })
    print("\n[시뮬레이션 데이터]")
    print(results_df)

---

## 1.5 LLM 감성 분석 결과 시각화

In [ ]:
# 감성 분포 시각화
sentiment_counts = results_df['sentiment'].value_counts()

# 색상 정의
colors = {'긍정': '#2ecc71', '부정': '#e74c3c', '중립': '#95a5a6'}

fig = px.pie(
    values=sentiment_counts.values,
    names=sentiment_counts.index,
    title='LLM 감성 분석 결과 분포',
    color=sentiment_counts.index,
    color_discrete_map=colors,
    hole=0.4
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label+value',
    textfont_size=14
)

fig.update_layout(
    width=600,
    height=450
)

fig.show()

---

# Part 2: 임베딩 기반 감성 분석

---

## 2.1 텍스트 임베딩 개념

### 임베딩이란?

**임베딩(Embedding)**은 텍스트를 고차원 벡터(숫자 배열)로 변환하는 것입니다.

### 왜 임베딩을 사용하나요?

| 특징 | 설명 |
|------|------|
| 의미 표현 | 비슷한 의미의 텍스트는 가까운 벡터 |
| 수치 연산 | 텍스트 간 유사도 계산 가능 |
| 범용성 | 검색, 분류, 추천 등 다양한 활용 |

### 예시

```
"좋아요" → [0.2, 0.8, 0.1, ...] (768차원 또는 3072차원)
"최고예요" → [0.25, 0.75, 0.15, ...] (비슷한 벡터)
"싫어요" → [-0.3, 0.1, 0.5, ...] (다른 벡터)
```

---

## 2.2 Google Gemini Embedding API

### gemini-embedding-001 특징

- **차원**: 3072 (기본), `output_dimensionality`로 축소 가능
- **지원 언어**: 100+ 언어 (한국어 포함)
- **최대 입력**: 2,048 토큰
- **무료 한도**: 월 100만 토큰

In [ ]:
def get_embedding_gemini(text, dimension=256):
    """
    Google Gemini API로 텍스트 임베딩 생성
    
    Parameters:
    - text: 텍스트 (문자열 또는 리스트)
    - dimension: 출력 차원 (기본 256, 최대 3072)
    
    Returns:
    - numpy array: 임베딩 벡터
    """
    if not client:
        return None
    
    result = client.models.embed_content(
        model="gemini-embedding-001",
        contents=text,
        config=types.EmbedContentConfig(output_dimensionality=dimension)
    )
    
    # 단일 텍스트면 첫 번째 임베딩만 반환
    if isinstance(text, str):
        return np.array(result.embeddings[0].values)
    else:
        return np.array([emb.values for emb in result.embeddings])


# 테스트
if client:
    test_text = "이 제품 정말 좋아요!"
    embedding = get_embedding_gemini(test_text, dimension=256)
    
    print(f"텍스트: {test_text}")
    print(f"임베딩 차원: {len(embedding)}")
    print(f"임베딩 (처음 10개): {embedding[:10].round(4)}")
else:
    print("API가 설정되지 않았습니다.")

---

## 2.3 sentence-transformers (로컬 임베딩)

### 장점

- **무료**: API 비용 없음
- **오프라인**: 인터넷 연결 불필요
- **빠름**: 로컬 처리로 지연 시간 없음

### 추천 모델

| 모델명 | 언어 | 차원 | 특징 |
|--------|------|------|------|
| `jhgan/ko-sbert-nli` | 한국어 | 768 | 한국어 특화 |
| `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` | 다국어 | 384 | 경량, 빠름 |
| `sentence-transformers/all-MiniLM-L6-v2` | 영어 | 384 | 고성능 |

In [ ]:
# sentence-transformers 설치 (최초 1회)
# !pip install sentence-transformers

In [ ]:
# sentence-transformers 로드
try:
    from sentence_transformers import SentenceTransformer
    
    # 한국어 모델 로드 (처음 실행 시 다운로드)
    local_model = SentenceTransformer('jhgan/ko-sbert-nli')
    local_embedding_available = True
    print("sentence-transformers 로드 완료")
except ImportError:
    local_embedding_available = False
    print("sentence-transformers 미설치. pip install sentence-transformers 로 설치하세요.")

In [ ]:
def get_embedding_local(text):
    """
    sentence-transformers로 로컬 임베딩 생성
    
    Parameters:
    - text: 텍스트 (문자열 또는 리스트)
    
    Returns:
    - numpy array: 임베딩 벡터
    """
    if not local_embedding_available:
        return None
    
    return local_model.encode(text)


# 테스트
if local_embedding_available:
    test_texts = [
        "이 제품 정말 좋아요!",
        "최고의 선택이었어요.",
        "완전 실망입니다."
    ]
    
    embeddings = get_embedding_local(test_texts)
    
    print("로컬 임베딩 생성 결과")
    print(f"임베딩 shape: {embeddings.shape}")
    for i, text in enumerate(test_texts):
        print(f"\n{text}")
        print(f"  임베딩 (처음 5개): {embeddings[i][:5].round(4)}")
else:
    print("sentence-transformers가 설치되지 않았습니다.")

---

## 2.4 코사인 유사도 기반 감성 분류

### 원리

1. **참조 임베딩 생성**: 긍정/부정 대표 문장의 임베딩
2. **대상 임베딩 생성**: 분류할 텍스트의 임베딩
3. **유사도 계산**: 코사인 유사도로 어느 쪽에 더 가까운지 측정
4. **분류**: 유사도가 높은 쪽으로 분류

### 코사인 유사도

```
cos(A, B) = (A · B) / (||A|| × ||B||)
```

- **1**: 완전히 동일한 방향 (매우 유사)
- **0**: 직교 (관련 없음)
- **-1**: 반대 방향 (완전히 다름)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

class EmbeddingSentimentClassifier:
    """
    임베딩 기반 감성 분류기
    """
    
    def __init__(self, embedding_func):
        """
        Parameters:
        - embedding_func: 임베딩 생성 함수
        """
        self.get_embedding = embedding_func
        self.reference_embeddings = {}
    
    def fit(self, positive_texts, negative_texts, neutral_texts=None):
        """
        참조 임베딩 생성
        
        Parameters:
        - positive_texts: 긍정 예제 리스트
        - negative_texts: 부정 예제 리스트
        - neutral_texts: 중립 예제 리스트 (선택)
        """
        # 긍정 임베딩 (평균)
        pos_embs = self.get_embedding(positive_texts)
        self.reference_embeddings['긍정'] = np.mean(pos_embs, axis=0)
        
        # 부정 임베딩 (평균)
        neg_embs = self.get_embedding(negative_texts)
        self.reference_embeddings['부정'] = np.mean(neg_embs, axis=0)
        
        # 중립 임베딩 (선택)
        if neutral_texts:
            neu_embs = self.get_embedding(neutral_texts)
            self.reference_embeddings['중립'] = np.mean(neu_embs, axis=0)
        
        print(f"참조 임베딩 생성 완료: {list(self.reference_embeddings.keys())}")
    
    def predict(self, text):
        """
        텍스트 감성 분류
        
        Returns:
        - dict: 분류 결과, 각 카테고리별 유사도
        """
        # 대상 임베딩
        if isinstance(text, str):
            text_emb = self.get_embedding(text).reshape(1, -1)
        else:
            text_emb = self.get_embedding(text)
        
        # 각 참조와 유사도 계산
        similarities = {}
        for label, ref_emb in self.reference_embeddings.items():
            sim = cosine_similarity(text_emb, ref_emb.reshape(1, -1))[0][0]
            similarities[label] = sim
        
        # 가장 유사한 카테고리
        predicted = max(similarities, key=similarities.get)
        
        return {
            'text': text if isinstance(text, str) else text[0],
            'prediction': predicted,
            'similarities': similarities
        }
    
    def predict_batch(self, texts):
        """여러 텍스트 일괄 분류"""
        results = []
        for text in texts:
            results.append(self.predict(text))
        return results

In [ ]:
# 임베딩 함수 선택 (API 또는 로컬)
if local_embedding_available:
    embedding_func = get_embedding_local
    print("로컬 임베딩 (sentence-transformers) 사용")
elif client:
    embedding_func = get_embedding_gemini
    print("Google Gemini Embedding API 사용")
else:
    embedding_func = None
    print("사용 가능한 임베딩 방법이 없습니다.")

In [ ]:
if embedding_func:
    # 분류기 초기화
    classifier = EmbeddingSentimentClassifier(embedding_func)
    
    # 참조 텍스트 정의
    positive_examples = [
        "정말 좋아요! 만족합니다.",
        "최고의 제품이에요!",
        "완전 추천해요. 훌륭합니다.",
        "기대 이상이에요. 감사합니다.",
        "품질이 좋고 배송도 빨라요."
    ]
    
    negative_examples = [
        "별로예요. 실망입니다.",
        "최악이에요. 돈 아까워요.",
        "환불 받고 싶어요.",
        "품질이 너무 안 좋아요.",
        "다시는 안 살 거예요."
    ]
    
    neutral_examples = [
        "그냥 그래요.",
        "보통이에요.",
        "무난합니다.",
        "특별히 좋지도 나쁘지도 않아요.",
        "가격 대비 적당해요."
    ]
    
    # 참조 임베딩 생성
    classifier.fit(positive_examples, negative_examples, neutral_examples)
else:
    print("임베딩 함수가 없어 분류기를 생성할 수 없습니다.")

In [ ]:
# 테스트
if embedding_func:
    test_reviews = [
        "이 제품 진짜 대박이에요! 너무 만족해요.",
        "기대했는데 완전 실망이에요.",
        "괜찮아요. 나쁘지 않아요.",
        "배송은 빨랐는데 품질이 아쉬워요.",
        "가성비 최고! 재구매 의사 있어요."
    ]
    
    print("임베딩 기반 감성 분류 결과")
    print("=" * 70)
    
    for review in test_reviews:
        result = classifier.predict(review)
        print(f"\n리뷰: {review}")
        print(f"  분류: {result['prediction']}")
        print(f"  유사도: 긍정={result['similarities']['긍정']:.3f}, "
              f"부정={result['similarities']['부정']:.3f}, "
              f"중립={result['similarities']['중립']:.3f}")
else:
    print("임베딩 함수가 없어 테스트를 건너뜁니다.")

### 💡 실무 예시: 임베딩 시각화 (PCA)

In [ ]:
from sklearn.decomposition import PCA

if embedding_func:
    # 모든 텍스트 임베딩
    all_texts = positive_examples + negative_examples + neutral_examples + test_reviews
    all_labels = (['긍정'] * len(positive_examples) + 
                  ['부정'] * len(negative_examples) + 
                  ['중립'] * len(neutral_examples) +
                  ['테스트'] * len(test_reviews))
    
    all_embeddings = embedding_func(all_texts)
    
    # PCA로 2차원 축소
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(all_embeddings)
    
    # 시각화
    vis_df = pd.DataFrame({
        'x': embeddings_2d[:, 0],
        'y': embeddings_2d[:, 1],
        'text': all_texts,
        'label': all_labels
    })
    
    fig = px.scatter(
        vis_df,
        x='x', y='y',
        color='label',
        hover_data=['text'],
        title='텍스트 임베딩 시각화 (PCA)',
        color_discrete_map={'긍정': '#2ecc71', '부정': '#e74c3c', '중립': '#95a5a6', '테스트': '#3498db'}
    )
    
    fig.update_traces(marker=dict(size=12))
    fig.update_layout(width=800, height=600)
    fig.show()
else:
    print("임베딩 함수가 없어 시각화를 건너뜁니다.")

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. Zero-shot 프롬프트 작성하기 ⭐

**문제**: 텍스트의 감성을 '긍정', '부정', '중립' 중 하나로 분류하는 Zero-shot 프롬프트를 작성하세요.

```python
text = "이 영화 정말 재미없어요. 시간 낭비였어요."
```

**요구사항**: 감성만 한 단어로 답하도록 프롬프트 작성

In [ ]:
text = "이 영화 정말 재미없어요. 시간 낭비였어요."

# 여기에 프롬프트를 작성하세요
prompt = """

"""

### Q2. Few-shot 예제 구성하기 ⭐⭐

**문제**: 음식점 리뷰 감성 분류를 위한 Few-shot 예제를 5개 작성하세요.

**요구사항**:
- 긍정 2개, 부정 2개, 중립 1개
- 실제 음식점 리뷰 스타일

In [ ]:
# 여기에 예제를 작성하세요
food_examples = [
    # ("리뷰 텍스트", "감성"),
]

### Q3. 코사인 유사도 계산하기 ⭐⭐

**문제**: 두 벡터 A와 B의 코사인 유사도를 직접 계산하는 함수를 작성하세요.

```python
A = np.array([1, 2, 3])
B = np.array([4, 5, 6])
```

**힌트**: `cos(A, B) = (A · B) / (||A|| × ||B||)`

In [ ]:
def cosine_similarity_manual(A, B):
    """
    코사인 유사도 수동 계산
    """
    # 여기에 코드를 작성하세요
    pass

A = np.array([1, 2, 3])
B = np.array([4, 5, 6])

# 테스트

### Q4. 5점 척도 감성 분류 프롬프트 ⭐⭐⭐

**문제**: LLM이 1~5점으로 감성을 평가하고 이유도 설명하는 프롬프트를 작성하세요.

```python
text = "배송은 빨랐는데 제품 품질이 기대에 못 미쳐요."
```

**출력 형식**:
```
점수: 3
이유: 배송은 좋았으나 품질에 불만족
```

In [ ]:
text = "배송은 빨랐는데 제품 품질이 기대에 못 미쳐요."

# 여기에 프롬프트를 작성하세요

### Q5. 측면(Aspect) 기반 감성 분석 프롬프트 ⭐⭐⭐

**문제**: 리뷰에서 여러 측면(배송, 품질, 가격 등)별로 감성을 추출하는 프롬프트를 작성하세요.

```python
text = "배송은 빠르고 좋았어요. 근데 품질이 별로고 가격도 비싸네요."
```

**기대 출력**:
```
- 배송: 긍정
- 품질: 부정
- 가격: 부정
```

In [ ]:
text = "배송은 빠르고 좋았어요. 근데 품질이 별로고 가격도 비싸네요."

# 여기에 프롬프트를 작성하세요

### Q6. 배치 감성 분석 함수 개선 ⭐⭐⭐

**문제**: 배치 감성 분석 함수에 진행률 표시와 에러 처리를 추가하세요.

**요구사항**:
- 진행률 표시 (예: "3/10 완료")
- API 에러 시 재시도 (최대 3회)
- 실패한 항목은 'error'로 표시

In [ ]:
def analyze_batch_improved(texts, max_retries=3):
    """
    개선된 배치 감성 분석 함수
    """
    # 여기에 코드를 작성하세요
    pass

### Q7. 임베딩 기반 감성 강도 측정 ⭐⭐⭐⭐

**문제**: 코사인 유사도를 활용하여 -1(매우 부정) ~ +1(매우 긍정) 사이의 연속적인 감성 점수를 계산하는 함수를 작성하세요.

**힌트**: `점수 = 긍정 유사도 - 부정 유사도`

In [ ]:
def calculate_sentiment_score(text, classifier):
    """
    임베딩 기반 감성 점수 계산 (-1 ~ +1)
    """
    # 여기에 코드를 작성하세요
    pass

### Q8. LLM vs 임베딩 비교 분석 ⭐⭐⭐⭐

**문제**: 동일한 10개 리뷰에 대해 LLM(Few-shot)과 임베딩 방식으로 감성을 분류하고, 결과를 비교하는 시각화를 만드세요.

**요구사항**:
- 10개 리뷰 분석
- 두 방식의 일치율 계산
- 불일치하는 리뷰 표시

In [ ]:
comparison_reviews = [
    "완전 만족해요! 최고의 구매였어요.",
    "돈 아까워요. 실망입니다.",
    "그냥 그래요. 보통이에요.",
    "배송은 빨랐는데 품질이...",
    "가성비 좋아요! 추천합니다.",
    "기대했는데 별로네요.",
    "나쁘지 않아요.",
    "최악이에요. 환불했어요.",
    "디자인 예쁘고 기능도 좋아요.",
    "가격만 좀 저렴하면 좋겠어요."
]

# 여기에 코드를 작성하세요

### Q9. 종합 감성 분석 파이프라인 ⭐⭐⭐⭐⭐

**문제**: 다음 기능을 모두 갖춘 종합 감성 분석 클래스를 작성하세요.

**요구사항**:
1. LLM(Zero-shot/Few-shot) + 임베딩 방식 모두 지원
2. 앙상블 분류 (두 방식 결과 종합)
3. 신뢰도 점수 산출
4. 배치 처리 지원

In [ ]:
class AdvancedSentimentAnalyzer:
    """
    종합 감성 분석기
    """
    
    def __init__(self):
        # 여기에 코드를 작성하세요
        pass
    
    def analyze(self, text, method='ensemble'):
        """
        텍스트 감성 분석
        
        Parameters:
        - text: 분석할 텍스트
        - method: 'llm', 'embedding', 'ensemble'
        """
        pass
    
    def analyze_batch(self, texts, method='ensemble'):
        """배치 분석"""
        pass

### Q10. 감성 분석 대시보드 ⭐⭐⭐⭐⭐

**문제**: 다음 데이터를 분석하여 종합 대시보드를 만드세요.

**분석 항목**:
1. 전체 감성 분포 (파이 차트)
2. 방법별 분류 결과 비교 (막대 차트)
3. 임베딩 시각화 (산점도)
4. 감성 점수 분포 (히스토그램)

**요구사항**:
- `make_subplots`로 2x2 레이아웃
- 인사이트 3개 이상 도출

In [ ]:
dashboard_reviews = [
    "정말 좋아요! 만족합니다.",
    "별로예요. 실망입니다.",
    "그냥 그래요.",
    "최고의 선택이었어요!",
    "돈 아까워요.",
    "가격 대비 괜찮아요.",
    "다시는 안 살 거예요.",
    "추천합니다!",
    "보통이에요.",
    "완전 실망이에요.",
    "기대 이상이에요!",
    "그냥 무난해요."
]

# 여기에 코드를 작성하세요

---

## 📊 학습 정리

### Part 1: LLM 기반 감성 분석 핵심 요약

| 개념 | 설명 | 활용 |
|------|------|------|
| Zero-shot | 예제 없이 바로 분류 | 빠른 프로토타이핑 |
| Few-shot | 몇 개 예제로 학습 | 도메인 특화 분류 |
| 프롬프트 엔지니어링 | 효과적인 지시문 작성 | 정확도 향상 |
| 배치 처리 | 여러 텍스트 일괄 분석 | 대량 데이터 처리 |

### Part 2: 임베딩 기반 감성 분석 핵심 요약

| 개념 | 설명 | 활용 |
|------|------|------|
| 텍스트 임베딩 | 텍스트를 벡터로 변환 | 의미 기반 분석 |
| Gemini Embedding | Google API 임베딩 | 고품질, 다국어 |
| sentence-transformers | 로컬 임베딩 | 무료, 오프라인 |
| 코사인 유사도 | 벡터 간 유사도 측정 | 감성 분류 |

### 💡 실무 팁

1. **Zero-shot 먼저**: 빠르게 프로토타입, 성능 부족 시 Few-shot
2. **도메인 예제**: Few-shot에 도메인 특화 예제 사용
3. **임베딩 캐싱**: 동일 텍스트 재분석 시 임베딩 저장해두기
4. **앙상블 활용**: LLM + 임베딩 결과 종합으로 신뢰도 향상
5. **비용 관리**: API 호출 횟수 최소화, 로컬 모델 활용
6. **배치 처리**: 대량 데이터는 배치로 효율적 처리
7. **시각화 필수**: 분석 결과는 항상 시각화로 검증